In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:

#loading data and fitting follows:
Xtrain=[]
Ytrain=[]
np.random.seed(1234567)
Xtrain=np.loadtxt('training_data/Xtrain_big.csv',delimiter=',')
Ytrain=np.loadtxt('training_data/YtrainR_big.csv',delimiter=',')
#Xtrain=Xtrain[:12000]
#Xtrain=np.transpose(Xtrain)
print(Xtrain.shape)

In [ ]:
img_size=2000
#Xtrain_N=[]
#[Xtrain_N.append(Xtrain[i]/np.abs(Xtrain[i][np.argmin(Xtrain[i])])) for i in range(0,3231)]
#Xtrain=np.array(Xtrain_N).reshape(3231,12000,1)
arr=np.arange(0,4500,1)
np.random.shuffle(arr)
Xtrain=np.array([Xtrain[i] for i in arr])
Ytrain=np.array([Ytrain[i] for i in arr])
#Xtrain=Xtrain[0:1500]
#Ytrain=Ytrain[0:1500]
print(Xtrain.shape)


In [ ]:
def sampling(mu_log_variance):
    mu, log_variance = mu_log_variance
    epsilon = keras.backend.random_normal(shape=keras.backend.shape(mu), mean=0.0, stddev=1.0)
    random_sample = mu + keras.backend.exp(log_variance/2) * epsilon
    return random_sample

In [ ]:
#model defined here
inp = keras.layers.Input(shape=(img_size, 1), name="encoder_input")
#leaky reLU has been added as a separate layer than define as an activation
x1 = keras.layers.Conv1D(filters=1, kernel_size=5, padding="same", strides=1,name="encoder_conv_1")(inp)
x1 = keras.layers.BatchNormalization(name="encoder_norm_1")(x1)
x1 = keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(x1)

x1 = keras.layers.Conv1D(filters=16, kernel_size=5, padding="same", strides=1, name="encoder_conv_2")(x1)
x1 = keras.layers.BatchNormalization(name="encoder_norm_2")(x1)
x1 = keras.layers.LeakyReLU(name="encoder_leakyrelu_2")(x1)

x1 = keras.layers.Conv1D(filters=32, kernel_size=5, padding="same", strides=1, name="encoder_conv_3")(x1)
x1 = keras.layers.BatchNormalization(name="encoder_norm_3")(x1)
bp_lay_1 = keras.layers.LeakyReLU(name="encoder_leakyrelu_3")(x1)
#flatten the layers in encoder
shape_before_flatten = keras.backend.int_shape(bp_lay_1)[1:]
x2 = keras.layers.Flatten(name="flat_1")(bp_lay_1)

latent_space_dim = 8
#declare a mean and variance for the distribution
encoder_mu = keras.layers.Dense(units=latent_space_dim, name="encoder_mu")(x2)
encoder_log_variance = keras.layers.Dense(units=latent_space_dim, name="encoder_log_variance")(x2)
encoder_op = keras.layers.Lambda(sampling, name="encoder_output")([encoder_mu, encoder_log_variance])

encoder = keras.models.Model(inp, encoder_op, name="encoder_model")
encoder.summary()


In [ ]:
#inp2 = keras.layers.Input(shape=shape_before_flatten, name="encoder_input")
x3 = keras.layers.AvgPool1D(2,strides=2,name='red_comp')(bp_lay_1)
x3 = keras.layers.Flatten(name="flat_2")(x3)
x3 = keras.layers.Dense(128, activation='relu')(x3)
x3 = keras.layers.Dropout(0.1)(x3)
x3 = keras.layers.Dense(32, activation='relu')(x3)
x3 = keras.layers.Dropout(0.1)(x3)
x3 = keras.layers.Dense(8, activation='relu')(x3)
#x3 = keras.Model(inputs=inp, outputs=x3)
#x3 = keras.layers.concatenate([encoder_mu, encoder_log_variance, x3.output])
out = keras.layers.Dense(2, activation='softmax')(x3)

full_model = keras.models.Model(inp,out, name="classifier")

In [ ]:
encoder.load_weights('encoding_simpler_moreD.h5')
for l1,l2 in zip(full_model.layers[:11],encoder.layers[0:11]):
    l1.set_weights(l2.get_weights())

print(full_model.get_weights()[0])
print(encoder.get_weights()[0])

In [ ]:
for layer in full_model.layers[0:11]:
    layer.trainable = False

full_model.summary()
full_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
classify_train = full_model.fit(Xtrain, Ytrain, batch_size=128,epochs=50,verbose=1,validation_split=0.2,shuffle=True,callbacks=[es_callback])

In [ ]:
full_model.save_weights('autoencoder_classification.h5')


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
test_loss, test_acc = full_model.evaluate(np.array(Xtrain), np.array(Ytrain))
print('Test accuracy:', test_acc)
Ypred_raw=full_model.predict(np.array(Xtrain))
Ypred=np.argmax(Ypred_raw, axis=1)
Ytest_new=np.argmax(Ytrain,axis=1)
cm = confusion_matrix(Ytest_new, Ypred)
print(cm)